In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
import seaborn as sns
import json 
 

#%matplotlib inline
plt.style.use('Solarize_Light2')

import warnings
warnings.filterwarnings("ignore")
plt.rcParams["figure.figsize"] = (10,6)

# Load Data

In [2]:
%%time
train=pd.read_feather("train_downcast")
train_weather=pd.read_feather("downcast_train_weather")
farm=pd.read_feather("downcast_farm")

Wall time: 292 ms


In [3]:
test=pd.read_feather("downcast_test")
test_weather=pd.read_feather("downcast_test_weather")

In [4]:
train_weather.rename(columns = {'timestamp':'date'}, inplace = True)

In [5]:
test_weather.rename(columns = {'timestamp':'date'}, inplace = True)

In [6]:
train_weather

,date,deidentified_location,temp_obs,cloudiness,wind_direction,dew_temp,pressure_sea_level,precipitation,wind_speed
0,2016-01-01 00:00:00,location 7369,25.000000,6.0,0.0,20.000000,1019.5,NaN,0.000000
1,2016-01-01 01:00:00,location 7369,24.406250,NaN,70.0,21.093750,1020.0,-1.0,1.500000
2,2016-01-01 02:00:00,location 7369,22.796875,2.0,0.0,21.093750,1020.0,0.0,0.000000
3,2016-01-01 03:00:00,location 7369,21.093750,2.0,0.0,20.593750,1020.0,0.0,0.000000
4,2016-01-01 04:00:00,location 7369,20.000000,2.0,250.0,20.000000,1020.0,-1.0,2.599609
...,...,...,...,...,...,...,...,...,...
139768,2016-12-31 19:00:00,location 5150,3.000000,NaN,180.0,-8.000000,NaN,NaN,5.699219
139769,2016-12-31 20:00:00,location 5150,2.800781,2.0,180.0,-8.898438,1007.5,NaN,7.699219
139770,2016-12-31 21:00:00,location 5150,2.800781,NaN,180.0,-7.199219,1007.5,NaN,5.101562
139771,2016-12-31 22:00:00,location 5150,2.199219,NaN,170.0,-6.699219,1008.0,NaN,4.601562


In [18]:
24*366*16

140544

In [16]:
main_train=train.merge(farm,on="farm_id",how="left").merge(train_weather,on=["deidentified_location","date"],how="left")

In [12]:
main_test=test.merge(farm,on="farm_id",how="left").merge(test_weather,on=["deidentified_location","date"],how="left")

In [25]:
main_train[(main_train["farm_id"]=="fid_122174")&(main_train["ingredient_type"]=="ing_y")]

,date,farm_id,ingredient_type,yield,operations_commencing_year,num_processing_plants,farm_area,farming_company,deidentified_location,temp_obs,cloudiness,wind_direction,dew_temp,pressure_sea_level,precipitation,wind_speed
1512,2016-01-01 00:00:00,fid_122174,ing_y,937.500000,NaN,NaN,10874.667969,Dole Food Company,location 2532,-8.296875,8.0,250.0,-12.203125,1023.5,NaN,4.101562
1513,2016-01-01 00:00:00,fid_122174,ing_y,937.500000,NaN,NaN,7411.987305,Wayne Farms,location 2532,-8.296875,8.0,250.0,-12.203125,1023.5,NaN,4.101562
3851,2016-01-01 01:00:00,fid_122174,ing_y,937.500000,NaN,NaN,10874.667969,Dole Food Company,location 2532,-8.296875,NaN,240.0,-11.703125,1023.5,-1.0,4.101562
3852,2016-01-01 01:00:00,fid_122174,ing_y,937.500000,NaN,NaN,7411.987305,Wayne Farms,location 2532,-8.296875,NaN,240.0,-11.703125,1023.5,-1.0,4.101562
6190,2016-01-01 02:00:00,fid_122174,ing_y,933.593994,NaN,NaN,10874.667969,Dole Food Company,location 2532,-7.800781,NaN,240.0,-11.101562,1023.0,-1.0,3.099609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20597232,2016-12-31 21:00:00,fid_122174,ing_y,1312.500000,NaN,NaN,7411.987305,Wayne Farms,location 2532,-4.398438,0.0,240.0,-11.703125,1009.5,0.0,4.101562
20599643,2016-12-31 22:00:00,fid_122174,ing_y,1265.619995,NaN,NaN,10874.667969,Dole Food Company,location 2532,-5.000000,0.0,230.0,-11.703125,1010.0,0.0,4.601562
20599644,2016-12-31 22:00:00,fid_122174,ing_y,1265.619995,NaN,NaN,7411.987305,Wayne Farms,location 2532,-5.000000,0.0,230.0,-11.703125,1010.0,0.0,4.601562
20602053,2016-12-31 23:00:00,fid_122174,ing_y,1277.339966,NaN,NaN,10874.667969,Dole Food Company,location 2532,-5.601562,4.0,240.0,-11.101562,1010.0,0.0,3.599609


In [18]:
for i in tqdm(main_train["ingredient_type"].unique()):
    df=main_train[main_train["ingredient_type"]==i]
    df.to_csv(f"train_{i}.csv",index=False)

  0%|          | 0/4 [00:00<?, ?it/s]

In [19]:
for i in tqdm(main_test["ingredient_type"].unique()):
    df=main_test[main_test["ingredient_type"]==i]
    df.to_csv(f"test_{i}.csv",index=False)

  0%|          | 0/4 [00:00<?, ?it/s]

In [37]:
df.to_csv("train_ing_w.csv",index=False)

In [38]:
df=main_train[main_train["ingredient_type"]=='ing_x']
df.to_csv("train_ing_x.csv",index=False)

In [39]:
df=main_train[main_train["ingredient_type"]=='ing_y']
df.to_csv("train_ing_y.csv",index=False)

In [40]:
df=main_train[main_train["ingredient_type"]=='ing_z']
df.to_csv("train_ing_z.csv",index=False)

  0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
train_weather.describe(percentiles=[0.1,0.25,0.5,0.75,0.99])

,temp_obs,cloudiness,wind_direction,dew_temp,pressure_sea_level,precipitation,wind_speed
count,139718.000000,70600.0,133505.0,139660.000000,129155.0,89484.0,139469.000000
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.000000,0.0,NaN,0.000000,NaN,NaN,0.000000
min,-28.906250,0.0,0.0,-35.000000,968.0,-1.0,0.000000
10%,0.600098,0.0,10.0,-5.601562,1007.0,0.0,0.500000
25%,7.199219,0.0,80.0,0.600098,1012.0,0.0,2.099609
50%,15.000000,2.0,190.0,8.296875,1016.5,0.0,3.099609
75%,22.203125,4.0,280.0,14.398438,1021.0,0.0,5.000000
99%,35.593750,8.0,360.0,23.906250,1034.0,25.0,10.296875
max,47.187500,9.0,360.0,26.093750,1046.0,343.0,19.000000


In [6]:
for i in train_weather.columns:
    print(train_weather[i].value_counts())
    print("~"*100)

2016-07-02 00:00:00    16
2016-08-30 12:00:00    16
2016-08-31 03:00:00    16
2016-08-31 02:00:00    16
2016-08-31 01:00:00    16
                       ..
2016-02-15 20:00:00    11
2016-04-07 15:00:00    11
2016-03-23 00:00:00    10
2016-03-23 01:00:00    10
2016-01-01 00:00:00     9
Name: timestamp, Length: 8784, dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
location 5677    8784
location 7369    8784
location 2532    8783
location 5489    8783
location 5833    8783
location 1784    8782
location 565     8782
location 5290    8780
location 8421    8780
location 5410    8777
location 959     8763
location 6364    8755
location 7048    8755
location 4525    8614
location 868     8614
location 5150    8454
Name: deidentified_location, dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
 15.000000    2722
 24.406250    2200
 20.000000    2191
 10.000000    218

In [7]:
for i in test_weather.columns:
    print(test_weather[i].value_counts())
    print("~"*100)

2017-07-02 12:00:00    16
2017-08-21 06:00:00    16
2017-08-22 12:00:00    16
2017-08-22 11:00:00    16
2017-08-22 10:00:00    16
                       ..
2017-08-15 19:00:00    11
2017-08-15 20:00:00    11
2017-01-18 19:00:00    11
2017-09-18 17:00:00    11
2017-01-01 00:00:00     9
Name: timestamp, Length: 8760, dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
location 5489    8760
location 5677    8760
location 7369    8760
location 2532    8759
location 5290    8759
location 5410    8759
location 5833    8759
location 565     8757
location 959     8753
location 7048    8750
location 8421    8749
location 6364    8745
location 1784    8743
location 5150    8520
location 4525    8246
location 868     8246
Name: deidentified_location, dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
15.000000    2823
23.296875    2456
10.000000    2382
23.906250    2317
22

In [5]:
test_weather

,timestamp,deidentified_location,temp_obs,cloudiness,wind_direction,dew_temp,pressure_sea_level,precipitation,wind_speed
0,2017-01-01 00:00:00,location 7369,17.796875,4.0,100.0,11.703125,1021.5,NaN,3.599609
1,2017-01-01 01:00:00,location 7369,17.796875,2.0,130.0,12.796875,1022.0,0.0,3.099609
2,2017-01-01 02:00:00,location 7369,16.093750,0.0,140.0,12.796875,1022.0,0.0,3.099609
3,2017-01-01 03:00:00,location 7369,17.203125,0.0,140.0,13.296875,1022.0,0.0,3.099609
4,2017-01-01 04:00:00,location 7369,16.703125,2.0,130.0,13.296875,1022.5,0.0,2.599609
...,...,...,...,...,...,...,...,...,...
138820,2017-12-31 19:00:00,location 5150,-15.000000,NaN,310.0,-18.296875,1028.0,-1.0,5.101562
138821,2017-12-31 20:00:00,location 5150,-15.000000,NaN,310.0,-18.906250,1028.0,-1.0,6.199219
138822,2017-12-31 21:00:00,location 5150,-15.000000,NaN,290.0,-19.406250,1028.0,-1.0,6.699219
138823,2017-12-31 22:00:00,location 5150,-15.601562,NaN,290.0,-19.406250,1028.0,NaN,6.699219


In [6]:
train_weather.isnull().sum()/len(train_weather)*100

timestamp                 0.000000
deidentified_location     0.000000
temp_obs                  0.039350
cloudiness               49.489529
wind_direction            4.484414
dew_temp                  0.080845
pressure_sea_level        7.596603
precipitation            35.979052
wind_speed                0.217496
dtype: float64

In [7]:
farm.isnull().sum()/len(farm)*100

farm_id                        0.000000
operations_commencing_year    53.416149
num_processing_plants         75.500345
farm_area                      0.000000
farming_company                0.000000
deidentified_location          0.000000
dtype: float64

In [8]:
farm=farm.drop(["operations_commencing_year","num_processing_plants"],axis=1)

# Null-value-Imputation 

In [4]:
miss_col=["cloudiness","wind_direction","dew_temp","temp_obs",
"pressure_sea_level","precipitation","wind_speed"]

In [10]:
train_weather[miss_col]=train_weather[miss_col].fillna(0)

In [12]:
train

,date,farm_id,ingredient_type,yield
0,2016-01-01 00:00:00,fid_110884,ing_w,0.000000
1,2016-01-01 00:00:00,fid_90053,ing_w,0.000000
2,2016-01-01 00:00:00,fid_17537,ing_w,0.000000
3,2016-01-01 00:00:00,fid_110392,ing_w,0.000000
4,2016-01-01 00:00:00,fid_62402,ing_w,0.000000
...,...,...,...,...
20216095,2016-12-31 23:00:00,fid_41664,ing_w,8.750000
20216096,2016-12-31 23:00:00,fid_14529,ing_w,4.825000
20216097,2016-12-31 23:00:00,fid_46454,ing_w,0.000000
20216098,2016-12-31 23:00:00,fid_24103,ing_w,159.574997


In [13]:
test

,date,farm_id,ingredient_type,id
0,2017-01-01 00:00:00,fid_110884,ing_w,0
1,2017-01-01 00:00:00,fid_90053,ing_w,1
2,2017-01-01 00:00:00,fid_17537,ing_w,2
3,2017-01-01 00:00:00,fid_110392,ing_w,3
4,2017-01-01 00:00:00,fid_62402,ing_w,4
...,...,...,...,...
20848795,2017-12-25 07:00:00,fid_58769,ing_x,41696845
20848796,2017-12-25 07:00:00,fid_58769,ing_z,41696846
20848797,2017-12-25 07:00:00,fid_117010,ing_w,41696847
20848798,2017-12-25 07:00:00,fid_117010,ing_x,41696848


In [15]:
train_weather.rename(columns = {'timestamp':'date'}, inplace = True)

In [20]:
train_data=train.merge(farm,on="farm_id")

In [18]:
test_data=test.merge(farm,on="farm_id")

In [27]:
ingz_train=train_data[train_data["ingredient_type"]=="ing_z"]
ingz_test=test_data[test_data["ingredient_type"]=="ing_z"]

In [35]:
df=ingz_train

# Aggeragating 

In [39]:
# individual
df_ind = df.groupby(['date', 'farm_id'])[['yield']].mean()
df_ind.reset_index(inplace=True)
df_ind = df_ind.T.reset_index(drop=True).T
df_ind.columns = ['ds', 'unique_id', 'yield']

KeyboardInterrupt: 

In [38]:
#location_wise
df_loc = df.groupby(['date','deidentified_location'])[['yield']].sum()
df_loc.reset_index(inplace=True)
df_loc = df_loc.T.reset_index(drop=True).T
df_loc.columns = ['ds', 'unique_id', 'yield']
df_loc

,ds,unique_id,yield
0,2016-01-01 00:00:00,location 1784,37395.96875
1,2016-01-01 00:00:00,location 2532,0.0
2,2016-01-01 00:00:00,location 4525,7506.209961
3,2016-01-01 00:00:00,location 5150,7394.615234
4,2016-01-01 00:00:00,location 5290,0.0
...,...,...,...
140539,2016-12-31 23:00:00,location 7048,0.0
140540,2016-12-31 23:00:00,location 7369,0.0
140541,2016-12-31 23:00:00,location 8421,89.849701
140542,2016-12-31 23:00:00,location 868,1021.546997


In [ ]:
#total
df_tot = df.groupby(['date'])[['yield']].sum()
df_tot.reset_index(inplace=True)
df_tot['unique_id'] = 'Total'
df_tot.columns = ['ds', 'yield', 'unique_id' ]

In [ ]:
dfx= pd.concat([df_ind,df_loc,df_tot],axis = 0)
# format
dfx.columns = ['ds','unique_id', 'y']
dfx['ds'] = pd.to_datetime(dfx['ds'])
dfx

# Summing Matrix

In [ ]:
xset = set(dfx.unique_id)
list1 = list(xset)
list1.sort()

list2 = [f for f in list1 if f.count('_') == 1]
list2.sort()

S = np.zeros((len(list1), len(list2)))
S = pd.DataFrame(S); S.index = list1; S.columns = list2

# populate the summing matrix
S.loc['Total'] = 1
for xcol in list1[:-1]:
    fl = [f for f in list2 if xcol in f]
    S.loc[xcol][fl] = 1

In [ ]:
loc_farm=df.groupby("deidentified_location")["farm_id"].unique()
dic=dict(zip(list(loc_farm.index),list(loc_farm.values)))

In [ ]:
t=[]
for i in dic.keys():
    for j in dic[i]:
        t.append((i,j))
for k in t:
    S.loc[str(k[0])][str(k[1])]=1

In [ ]:
tags={}
tags["total"]=df_tot.unique_id.unique()
tags['total/location'] = df_loc.unique_id.unique()
tags['total/location/individual'] = df_ind.unique_id.unique()
tags

In [ ]:
dfx=dfx.sort_values(by=["ds","unique_id"])

<a id="section-zero"></a>
# Why are we here?

## Hierarchical time series

* time series in various application have a natural hierarchy (a level structure): 

[source](https://towardsdatascience.com/forecast-reconciliation-in-python-8203c16254b2)

![hierarchy_levels.png](attachment:f180129b-8501-4347-aa9e-983430e6e551.png)

* SC1 exclusively under C1, SC4 under C2 $\implies$ **hierarchical time series**

* non exclusive $\implies$ grouped time series (not naturally disaggregating in a unique hierarchical manner)

![grouped_levels.png](attachment:f703de32-0e80-4884-879c-7671e1bb8a59.png)

* additive across levels: in a manner consistent with the aggregation structure

\begin{equation}
T = C_1 + C_2  \\
C_1 = SC_1 + SC_2 + SC_3 \\
C_2 = SC_4 + SC_5
\end{equation}

* most time series methods focus on a particular level of aggregation, but fail to address drilling down to a more granular level.

* sum of lower level $\neq$ higher level $\implies$ incoherent forecast 

* enter HTS

* **not a methodology** (like ARIMA, ETS, Prophet) but a collection of techniques to achieve coherence

* hierarchy components can interact $\implies$ simultaneous modeling yields higher quality forecasts


Why are forecasts incoherent?

1. correct high level forecasts, issues at low levels

2. correct low level forecasts, issues with aggregation

3. combination of 1. and 2. 

## The setup

In order to ensure forecast coherence we need to break our hierarchical time series into components

### Base Forecasts

We build univariate forecasts at each individual level across the hierarchy.

### Summing matrix

* The summing matrix describes the hierarchical structure of the data

* columns = unique series on the bottom level

* rows = unique series across all levels

* values = 0/1 $\rightarrow$ which bottom-level (column) categories map to each of the hierarchies across all levels. Since picture $\gg$ 1000 words:

[source](https://towardsdatascience.com/forecast-reconciliation-in-python-8203c16254b2)

![summing_matrix.png](attachment:5d420c59-ff9f-4f4d-9bdd-8b9293df0106.png)


### Mapping Matrix

* multiply by summing matrix and based forecast $\implies$ coherent forecasts

* objective: reconcile the forecasts with minimal variance

* Varies per reconciliation method used 

In [21]:
for f in (["farm_id","ingredient_type","farming_company","deidentified_location"]):
    print(f)
    print(train_data[f].unique())
    print("~"*100)

farm_id
['fid_110884', 'fid_90053', 'fid_17537', 'fid_110392', 'fid_62402', ..., 'fid_81701', 'fid_46585', 'fid_116275', 'fid_60652', 'fid_19864']
Length: 1434
Categories (1434, object): ['fid_100053', 'fid_100107', 'fid_100135', 'fid_100249', ..., 'fid_99482', 'fid_99826', 'fid_99921', 'fid_99942']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ingredient_type
['ing_w', 'ing_x', 'ing_z', 'ing_y']
Categories (4, object): ['ing_w', 'ing_x', 'ing_y', 'ing_z']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
farming_company
['Obery Farms', 'Dole Food Company', 'Wayne Farms', 'Sanderson Farms', 'Other', ..., 'Foster Farms', 'Kansas Ville Farms', 'West Liberty Foods', 'Monogram Foods', 'Northland Organic Foods Corporation']
Length: 16
Categories (16, object): ['Del Monte Foods', 'Dole Food Company', 'Foster Farms', 'Freight Farms', ..., 'Southern Confederate Farms', 'Tyson Foods', 'Wayn

In [30]:
#data=data.sort_values(by=['farm_id',"date"])

In [28]:
#data=data.drop(["farm_area","farming_company"],axis=1)

In [32]:
#data.reset_index()

,index,date,farm_id,ingredient_type,yield,deidentified_location
0,4297678,2016-01-01 00:00:00,fid_100053,ing_w,37.040001,location 5290
1,4297679,2016-01-01 01:00:00,fid_100053,ing_w,37.119999,location 5290
2,4297680,2016-01-01 02:00:00,fid_100053,ing_w,37.779999,location 5290
3,4297681,2016-01-01 03:00:00,fid_100053,ing_w,36.150002,location 5290
4,4297682,2016-01-01 04:00:00,fid_100053,ing_w,38.660000,location 5290
...,...,...,...,...,...,...
20602894,2870163,2016-12-31 22:00:00,fid_99942,ing_x,124.778000,location 5489
20602895,2870164,2016-12-31 22:00:00,fid_99942,ing_z,35.168499,location 5489
20602896,2870165,2016-12-31 23:00:00,fid_99942,ing_w,79.309998,location 5489
20602897,2870166,2016-12-31 23:00:00,fid_99942,ing_x,72.623001,location 5489


In [36]:
for i in (["farm_id","ingredient_type","deidentified_location"]):
    data[i]=data[i].astype("object")

In [37]:
data["location_wise"]=data["farm_id"]+"_"+data["deidentified_location"]
data["ingredient_wise"]=data["location_wise"]+"_"+data["ingredient_type"]

In [43]:
sample=data["farm_id"].unique()[:10]

In [47]:
data[data["farm_id"]=="fid_100053"]["deidentified_location"].unique()

array(['location 5290'], dtype=object)

In [40]:
data['cnt'] = data.groupby('ingredient_wise').transform('count')
data

ValueError: Wrong number of items passed 6, placement implies 1